In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from LookGenerator.networks.losses import WassersteinLoss, GradientPenalty, FineGANLoss
from LookGenerator.datasets.encoder_decoder_datasets import EncoderDecoderDataset
from LookGenerator.networks.fine_gan import *
from LookGenerator.networks.clothes_feature_extractor import ClothAutoencoder
from LookGenerator.networks.trainer import WGANGPTrainer
from LookGenerator.networks_training.utils import check_path_and_creat
from LookGenerator.networks.utils import get_num_digits, save_model, load_model
import LookGenerator.datasets.transforms as custom_transforms

In [2]:
transform_input = transforms.Compose([
    transforms.Resize((256, 192)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

transform_real = transforms.Compose([
    transforms.Resize((256, 192)),
    custom_transforms.MinMaxScale()
])

In [3]:
batch_size_train = 24
pin_memory = True
num_workers = 6

In [4]:
train_dataset = EncoderDecoderDataset(
    image_dir=r"C:\Users\DenisovDmitrii\Desktop\forEncoderNew\train",
    transform_human=transform_input,
    transform_clothes=transform_input,
    transform_human_restored=transform_real
)

In [5]:
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size_train, shuffle=True, pin_memory=pin_memory, num_workers=num_workers
)

In [6]:
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.benchmark = True

In [7]:
# def _epoch_string(epoch, epoch_num):
#     num_digits_epoch_num = get_num_digits(epoch_num)
#     num_digits_epoch = get_num_digits(epoch)
#
#     epoch_string = "0"*(num_digits_epoch_num - num_digits_epoch) + str(epoch)
#     return epoch_string


In [8]:
# def fit(model, criterion, gradient_penalty, train_dl, device, epochs, g_lr, d_lr,
#         save_directory_generator, save_directory_discriminator, save_step=1):
#     model["discriminator"].train()
#     model["generator"].train()
#     torch.cuda.empty_cache()
#
#     # Losses & scores
#     losses_g = []
#     losses_d = []
#     real_scores = []
#     fake_scores = []
#
#     # Create optimizers
#     optimizer = {
#         "discriminator": torch.optim.Adam(model["discriminator"].parameters(),
#                                           lr=d_lr, betas=(0.5, 0.999)),
#         "generator": torch.optim.Adam(model["generator"].parameters(),
#                                       lr=g_lr, betas=(0.5, 0.999))
#     }
#
#     for epoch in range(epochs):
#         loss_d_per_epoch = []
#         loss_g_per_epoch = []
#         real_score_per_epoch = []
#         fake_score_per_epoch = []
#         model['discriminator'] = model['discriminator'].to(device)
#         model['generator'] = model['generator'].to(device)
#         for iteration, (input_images, real_images) in enumerate(tqdm(train_dl), 0):
#             input_images = input_images.to(device)
#             real_images = real_images.to(device)
#             # Train discriminator
#             # Clear discriminator gradients
#             optimizer["discriminator"].zero_grad()
#
#             real_images = real_images.to(device)
#
#             # Pass real images through discriminator
#             real_preds = model["discriminator"](real_images)
#             real_targets = torch.ones(real_images.shape[0], 1, device=device)
#             real_loss = criterion["discriminator"](real_preds, real_targets)
#             cur_real_score = torch.mean(real_preds).item()
#
#             # Generate fake images
#             fake_images = model["generator"](input_images)
#
#             # Pass fake images through discriminator
#             fake_targets = torch.ones(fake_images.shape[0], 1, device=device)
#             fake_preds = model["discriminator"](fake_images)
#             fake_loss = criterion["discriminator"](fake_preds, fake_targets)
#             cur_fake_score = torch.mean(fake_preds).item()
#             gp = gradient_penalty(model["discriminator"], real_images, fake_images, device)
#
#             real_score_per_epoch.append(cur_real_score)
#             fake_score_per_epoch.append(cur_fake_score)
#
#             # Update discriminator weights
#             loss_d = real_loss + fake_loss + 10.0 * gp
#             loss_d.backward()
#             optimizer["discriminator"].step()
#             loss_d_per_epoch.append(loss_d.item())
#
#             # Train generator
#             if iteration % 5 == 0:
#             # Clear generator gradients
#                 optimizer["generator"].zero_grad()
#
#                 # Generate fake images
#                 fake_images = model["generator"](input_images)
#
#                 # Try to fool the discriminator
#                 preds = model["discriminator"](fake_images)
#                 targets = torch.ones(real_images.shape[0], 1, device=device)
#                 loss_g = criterion["generator"](preds, targets, fake_images, real_images)
#
#                 # Update generator weights
#                 loss_g.backward()
#                 optimizer["generator"].step()
#                 loss_g_per_epoch.append(loss_g.item())
#
#                 losses_g.append(np.mean(loss_g_per_epoch))
#
#         # Record losses & scores
#         losses_d.append(np.mean(loss_d_per_epoch))
#         real_scores.append(np.mean(real_score_per_epoch))
#         fake_scores.append(np.mean(fake_score_per_epoch))
#
#         # Log losses & scores (last batch)
#         print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
#             epoch+1, epochs,
#             losses_g[-1], losses_d[-1], real_scores[-1], fake_scores[-1])
#         )
#
#         if (epoch + 1) % save_step == 0:
#             save_model(
#                 model["discriminator"].to('cpu'),
#                 path=f"{save_directory_discriminator}\\discriminator_epoch_{_epoch_string(epoch, epochs)}.pt"
#             )
#             save_model(
#                 model["generator"].to('cpu'),
#                 path=f"{save_directory_generator}\\generator_epoch_{_epoch_string(epoch, epochs)}.pt"
#             )
#
#     return losses_g, losses_d, real_scores, fake_scores

In [7]:
save_directory = r'C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\encoderGAN\weights\testBaseParam'
check_path_and_creat(save_directory)
save_directory_generator = os.path.join(save_directory, 'gen')
save_directory_discriminator = os.path.join(save_directory, 'discr')
check_path_and_creat(save_directory_generator)
check_path_and_creat(save_directory_discriminator)

True

In [8]:
clothes_feature_extractor = ClothAutoencoder(
    in_channels=3,
    out_channels=3,
    features=(8, 16, 32, 64),
    latent_dim_size=128,
    encoder_activation_func=nn.LeakyReLU(),
    decoder_activation_func=nn.ReLU()
)
clothes_feature_extractor = load_model(clothes_feature_extractor, r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\autoDegradation\weights\testClothes_L1Loss_4features\epoch_39.pt")

In [9]:
generator = EncoderDecoderGenerator(clothes_feature_extractor=clothes_feature_extractor, in_channels=3, out_channels=3, final_activation_func=nn.Sigmoid())
discriminator = Discriminator()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

wasserstein_criterion = WassersteinLoss()
gradient_penalty = GradientPenalty(discriminator, device=device)

criterion_generator = FineGANLoss(
    adversarial_criterion=wasserstein_criterion, adv_loss_weight=0.25,
    l1_criterion=True, l1_loss_weight=4,
    perceptual=True, perceptual_loss_weight=1, device=device
)
criterion_discriminator = WassersteinLoss()

print(device)

C:\Users\DenisovDmitrii\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DenisovDmitrii\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda


In [10]:
optimizer_generator = torch.optim.Adam(params=generator.parameters(), lr=0.001)
optimizer_discriminator = torch.optim.Adam(params=discriminator.parameters(), lr=0.001)

In [13]:
model = {
    "discriminator": discriminator.to(device),
    "generator": generator.to(device)
}

criterion = {
    "discriminator": criterion_discriminator.to(device),
    "generator": criterion_generator.to(device)
}

In [14]:
trainer = WGANGPTrainer(
    generator=generator,
    discriminator=discriminator,
    optimizer_generator=optimizer_generator,
    optimizer_discriminator=optimizer_discriminator,
    criterion_generator=criterion_generator,
    criterion_discriminator=criterion_discriminator,
    gradient_penalty=gradient_penalty,
    gp_weight=0.2,
    save_step=1,
    save_directory_discriminator=save_directory_discriminator,
    save_directory_generator=save_directory_generator,
    device=device,
    verbose=True
)


In [15]:
# history = fit(model=model,
#               criterion=criterion,
#               gradient_penalty=gradient_penalty,
#               train_dl=train_dataloader,
#               device=device,
#               epochs=10,
#               g_lr=0.0001,
#               d_lr=0.0001,
#               save_directory_generator=save_directory_generator,
#               save_directory_discriminator=save_directory_discriminator)

In [16]:
epoch_num = 20

In [17]:
history = trainer.train(
    train_dataloader=train_dataloader,
    epoch_num=epoch_num
)

start time 02-06-2023 15:35


  0%|          | 0/486 [00:00<?, ?it/s]C:\Users\DenisovDmitrii\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 486/486 [05:53<00:00,  1.38it/s]


Epoch 0 of 19, discriminator loss: 0.00251
Epoch 0 of 19, generator loss: 4.11509
Epoch end time 02-06-2023 15:41


100%|██████████| 486/486 [06:34<00:00,  1.23it/s]


Epoch 1 of 19, discriminator loss: 0.00000
Epoch 1 of 19, generator loss: 3.10050
Epoch end time 02-06-2023 15:47


...

100%|██████████| 486/486 [05:27<00:00,  1.48it/s]


Epoch 12 of 19, discriminator loss: 0.00000
Epoch 12 of 19, generator loss: 2.43512
Epoch end time 02-06-2023 16:53


100%|██████████| 486/486 [05:28<00:00,  1.48it/s]


Epoch 13 of 19, discriminator loss: 0.00000
Epoch 13 of 19, generator loss: 2.41783
Epoch end time 02-06-2023 16:58


 15%|█▍        | 72/486 [00:55<04:39,  1.48it/s] 

In [18]:
trainer.draw_history_plots()

In [19]:
trainer.save_history_plots(save_directory)

In [20]:
trainer.create_readme(save_directory)

In [21]:
test_dataset = EncoderDecoderDataset(
    image_dir=r"C:\Users\DenisovDmitrii\Desktop\forEncoderNew\val",
    transform_human=transform_input,
    transform_clothes=transform_input,
    transform_human_restored=transform_real
)

In [22]:
generator.to('cpu')
discriminator.to('cpu')
image, real_image = test_dataset[2]
image = image.unsqueeze(0)
#print(image)
print(image.shape)
image = generator(image)
imaged = discriminator(image)
image = transforms.ToPILImage()(image[0, :, :, :])
image.show()
print(imaged)
